# Todo: Run masked datasets
# 1. LFW masked
# 2. Custom masked dataset


In [ ]:
!pip install mtcnn

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
pip install git+https://github.com/paoloripamonti/face-recognition


  Cloning https://github.com/paoloripamonti/face-recognition to /tmp/pip-req-build-4i3swfyf
  Running command git clone -q https://github.com/paoloripamonti/face-recognition /tmp/pip-req-build-4i3swfyf
  Created wheel for facenet-face-recognition: filename=facenet_face_recognition-0.1-cp36-none-any.whl size=5969 sha256=ea039ac00e16d06ccd432d4e0a2a1ef1bedf8b8a8b0b76d743a3974830fb6717
  Stored in directory: /tmp/pip-ephem-wheel-cache-ssgrk0_x/wheels/b4/0b/30/4413165ce9aaebf21fd5920f82bfc94ab6cf8fdb3dd543bf12
Successfully built facenet-face-recognition


In [ ]:
import pickle

import numpy as np
from sklearn.metrics.pairwise import euclidean_distances


class EuclideanClassifier(object):
    """
    Euclidean distance classifier
    """

    def __init__(self):
        """
        Initialize classifier
        """
        self.X = []
        self.Y = []

    def fit(self, x_train, y_train):
        """
        Add encodings and labels into classifier
        :param x_train: encodings
        :param y_train: labels
        """
        if len(self.X) > 0 and len(self.Y) > 0:
            self.X = np.concatenate((self.X, x_train), axis=0)
            self.Y = self.Y + y_train
        else:
            self.X = x_train
            self.Y = y_train

    def predict(self, x_test):
        """
        Perform prediction from given encoding
        :param x_test: given encoding
        :return: dictionary with label and confidence (euclidean distance)
        """
        distances = euclidean_distances(np.expand_dims(x_test, axis=0), self.X)[0]
        idx = int(np.argmin(distances))
        max_val, min_val = 1, -1
        raw_confidence = max(1 - distances[idx], min_val)
        return {"person": self.Y[idx], "confidence": (raw_confidence - min_val) / (max_val - min_val)}

    def load(self, path):
        """
        Load classifier from pickle file
        :param path: path
        """
        database = pickle.load(open(path, "rb"))

        self.X = database["encodings"]
        self.Y = database["people"]

    def save(self, path):
        """
        Save classifier as pickle file
        :param path: path
        """

        database = {
            "encodings": self.X,
            "people": self.Y
        }

        pickle.dump(database, open(path, "wb"))


In [ ]:
FACE_SIZE = (160, 160)
INPUT_SHAPE = (160, 160, 3)
FACE_DETECTION_SIZE = (240, 200)
MIN_FACE_SIZE = (48, 48)
UNKNOWN_LABEL = "UNKNOWN"
FACE_CONFIDENCE = .9
DEFAULT_THRESHOLDS = [0.6, 0.9]
ALLOWED_IMAGE_TYPES = ('*.jpeg', '*.png', '*.jpg')

In [ ]:
import base64
import glob
import os
import time
import warnings
import random
import requests

import cv2
import imutils
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import tqdm
from keras_facenet import FaceNet
from mtcnn.mtcnn import MTCNN
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, roc_curve, precision_recall_curve, roc_auc_score, accuracy_score

warnings.filterwarnings("ignore")


class FaceRecognitionClass(object):
    """
    Face Recognition object class
    """

    def __init__(self):
        """
        Initialize Face Recognition model
        """
        # GRAPH
        if (tf.__version__) > '2.0':
          self.graph = tf.compat.v1.get_default_graph()
        else:
          self.graph = tf.get_default_graph()

        # Load Face Detector
        self.face_detector = MTCNN()

        # Load FaceNet
        self.facenet = FaceNet()

        # Euclidean Classifier
        self.clf = None

        tf.compat.v1.disable_eager_execution()

    def predict(self, path, threshold=None):
        """
        Find faces and recognize them, return predicted people into image
        :param path: Source image path
        :param threshold: cutoff threshold
        :return: Return predictions and images with rectangles drawn
        """
        if not self.clf:
            raise RuntimeError("No classifier found. Please load classifier")

        start_at = time.time()
        bounding_boxes = []
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = image.astype(np.uint8)
        for person, confidence, box in self.__predict__(image, threshold=threshold):
            # Draw rectangle with person name
            cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
            cv2.putText(image, person, (box[0], box[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0))

            bounding_boxes.append({
                "person": person,
                "confidence": confidence,
                "box": box,
            })

        # encode frame
        _, buffer = cv2.imencode('.jpg', image)

        return {
            "frame": base64.b64encode(buffer).decode('ascii'),
            "elapsed_time": (time.time() - start_at),
            "predictions": bounding_boxes

        }

    def __predict__(self, image, threshold=None):
        """
        Extract face and perform evaluation
        :param image: Source image
        :param threshold: decision threshold
        :return:  yield (person_id, person, confidence, box)
        """
        # Resize Image
        for encoding, face, box in self.face_encoding(image):
            # Check face size
            if (box[2] - box[0]) < MIN_FACE_SIZE[0] or \
                    (box[3] - box[1]) < MIN_FACE_SIZE[1]:
                yield (UNKNOWN_LABEL, 0.0, box)
            else:
                results = self.clf.predict(encoding)
                person, confidence = results["person"], results["confidence"]
                if threshold and confidence < threshold:
                    person = UNKNOWN_LABEL

                yield (person, confidence, box)

    def face_detection(self, image):
        """
        Face detection from source image
        :param image: Source image
        :return: extracted face and bounding box
        """
        image_to_detect = image.copy()

        # detect faces in the image
        for face_attributes in self.face_detector.detect_faces(image_to_detect):
            if face_attributes["confidence"] > FACE_CONFIDENCE:
                # extract the bounding box
                x1, y1, w, h = [max(point, 0) for point in face_attributes["box"]]
                x2, y2 = x1 + w, y1 + h

                face = image[y1:y2, x1:x2]
                # Align face
                face = FaceRecognitionClass.align_face(face_attributes, face.copy())

                yield (cv2.resize(face, FACE_SIZE), (x1, y1, x2, y2))

    def face_encoding(self, source_image):
        """
        Extract face encodings from image
        :param source_image: Source image
        :return: 512 encoding, face and bounding box
        """
        for face, box in self.face_detection(source_image):
            with self.graph.as_default():
                # Face encoding
                encoding = self.facenet.embeddings(np.expand_dims(face, axis=0))[0]

                yield (encoding, face, box)

    @staticmethod
    def align_face(face_attribute, image):
        if not face_attribute:
            return image
        # Get left and right eyes
        left_eye = face_attribute["keypoints"]["left_eye"]
        right_eye = face_attribute["keypoints"]["right_eye"]
        # Get distance between eyes
        d = math.sqrt(math.pow(right_eye[0] - left_eye[0], 2) + math.pow(right_eye[1] - left_eye[1], 2))
        a = left_eye[1] - right_eye[1]
        # get alpha degree
        alpha = (math.asin(a / d) * 180.0) / math.pi

        return imutils.rotate(image, -alpha)

    def load(self, path):
        """
        Load classifier from pickle file
        :param path: path
        """
        clf = EuclideanClassifier()
        clf.load(path)

        self.clf = clf

    def save(self, path):
        """
        Save classifier as pickle file
        :param path: path
        """
        self.clf.save(path)

    def fit(self, folder):
        """
        Fit classifier from directory.
        Directory must have this structure:
            Person 1:
                file.jpg
                ....
                file.jpg
            Person 2:
                file.jpg
                ...
                file.jpg
            ...
        :param folder: root folder path
        """
        # Initialize classifier
        clf = EuclideanClassifier()

        # Load all files
        files = []
        for ext in ALLOWED_IMAGE_TYPES:
            files.extend(glob.glob(os.path.join(folder, "*", ext), recursive=True))

        for path in tqdm.tqdm(files):
            # Load image
            print(path)
            image = cv2.imread(path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Get person name by folder
            person = os.path.split(os.path.split(path)[0])[1]

            # Get encoding
            for encoding, face, box in self.face_encoding(image):
                # Add to classifier
                clf.fit([encoding], [person])

        self.clf = clf

    def fit_from_dataframe(self, df, person_col="person", path_col="path"):
        """
        Fit classifier from dataframe.
        :param df: Pandas dataframe
        :param person_col: Dataframe column with person id
        :param path_col: Dataframe column with image path
        """
        # Initialize classifier
        clf = EuclideanClassifier()

        for index, row in df.iterrows():
            # Load image
            image = cv2.imread(row[path_col])
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Get person name by folder
            person = row[person_col]

            # Get encoding
            for encoding, face, box in self.face_encoding(image):
                # Add to classifier
                clf.fit([encoding], [person])

        self.clf = clf

In [ ]:
#Loading the Trainig Dataset

#prefix = '/content/gdrive/My Drive/Advanced Machine Learning/Project/lfw_masked/'

#To check with LFW custom masked dataset, uncomment the below line.
#prefix = '/content/gdrive/My Drive/Advanced Machine Learning/Project/'

dataset = []
for path in glob.iglob(os.path.join('/content/gdrive/MyDrive/Advanced Machine Learning/Project/lfw_funneled', "**", "*.jpg")):
#for path in glob.iglob(os.path.join(prefix+'lfw_train/', "**", "*.jpg")):
  print(path)
  person = path.split("/")[-2]
  person = path.split("/")[-2]
  dataset.append({"person":person, "path": path})
    
dataset = pd.DataFrame(dataset)
dataset = dataset.groupby("person").filter(lambda x: len(x) > 0)
dataset.head(60)
print(dataset.shape)

Streaming output truncated to the last 5000 lines.
/content/gdrive/MyDrive/Advanced Machine Learning/Project/lfw_funneled/Rachel_Hunter/Rachel_Hunter_0002.jpg
/content/gdrive/MyDrive/Advanced Machine Learning/Project/lfw_funneled/Rachel_Hunter/Rachel_Hunter_0001.jpg
/content/gdrive/MyDrive/Advanced Machine Learning/Project/lfw_funneled/Chip_Ganassi/Chip_Ganassi_0001.jpg
/content/gdrive/MyDrive/Advanced Machine Learning/Project/lfw_funneled/Alejandro_Gonzalez_Inarritu/Alejandro_Gonzalez_Inarritu_0001.jpg
/content/gdrive/MyDrive/Advanced Machine Learning/Project/lfw_funneled/Julie_Gerberding/Julie_Gerberding_0014.jpg
/content/gdrive/MyDrive/Advanced Machine Learning/Project/lfw_funneled/Julie_Gerberding/Julie_Gerberding_0004.jpg
/content/gdrive/MyDrive/Advanced Machine Learning/Project/lfw_funneled/Julie_Gerberding/Julie_Gerberding_0005.jpg
/content/gdrive/MyDrive/Advanced Machine Learning/Project/lfw_funneled/Julie_Gerberding/Julie_Gerberding_0008.jpg
/content/gdrive/MyDrive/Advanced Ma

In [ ]:
#Loading the testing dataset
test_dataset = []

for path in glob.iglob(os.path.join('/content/gdrive/MyDrive/Advanced Machine Learning/Project/mask_test/', "**", "*.jpg")):
#for path in glob.iglob(os.path.join('/content/gdrive/MyDrive/Advanced Machine Learning/Project/mask_eyewear/', "**", "*.jpg")):
#for path in glob.iglob(os.path.join(prefix+'lfw_test/', "**", "*.jpg")):
    person = path.split("/")[-2]
    test_dataset.append({"person":person, "path": path})

print(test_dataset) 
test_dataset = pd.DataFrame(test_dataset)
test_dataset = test_dataset.groupby("person").filter(lambda x: len(x) > 0)
test_dataset.head(60)


[{'person': 'Lara_Logan', 'path': '/content/gdrive/MyDrive/Advanced Machine Learning/Project/mask_test/Lara_Logan/Lara_Logan_0001.jpg'}, {'person': 'Conan_OBrien', 'path': '/content/gdrive/MyDrive/Advanced Machine Learning/Project/mask_test/Conan_OBrien/Conan_OBrien_0002.jpg'}, {'person': 'Donald_Keck', 'path': '/content/gdrive/MyDrive/Advanced Machine Learning/Project/mask_test/Donald_Keck/Donald_Keck_0001.jpg'}, {'person': 'Beyonce_Knowles', 'path': '/content/gdrive/MyDrive/Advanced Machine Learning/Project/mask_test/Beyonce_Knowles/Beyonce_Knowles_0001.jpg'}, {'person': 'John_Nash', 'path': '/content/gdrive/MyDrive/Advanced Machine Learning/Project/mask_test/John_Nash/John_Nash_0001.jpg'}, {'person': 'Melinda_Czink', 'path': '/content/gdrive/MyDrive/Advanced Machine Learning/Project/mask_test/Melinda_Czink/Melinda_Czink_0001.jpg'}, {'person': 'Tony_Clement', 'path': '/content/gdrive/MyDrive/Advanced Machine Learning/Project/mask_test/Tony_Clement/Tony_Clement_0001.jpg'}, {'person': 

,person,path
0,Lara_Logan,/content/gdrive/MyDrive/Advanced Machine Learn...
1,Conan_OBrien,/content/gdrive/MyDrive/Advanced Machine Learn...
2,Donald_Keck,/content/gdrive/MyDrive/Advanced Machine Learn...
3,Beyonce_Knowles,/content/gdrive/MyDrive/Advanced Machine Learn...
4,John_Nash,/content/gdrive/MyDrive/Advanced Machine Learn...
5,Melinda_Czink,/content/gdrive/MyDrive/Advanced Machine Learn...
6,Tony_Clement,/content/gdrive/MyDrive/Advanced Machine Learn...
7,Richard_Langille,/content/gdrive/MyDrive/Advanced Machine Learn...
8,Lloyd_Novick,/content/gdrive/MyDrive/Advanced Machine Learn...
9,Fran_Drescher,/content/gdrive/MyDrive/Advanced Machine Learn...


In [ ]:
!pip install git+https://github.com/paoloripamonti/face-recognition

  Cloning https://github.com/paoloripamonti/face-recognition to /tmp/pip-req-build-6dd2fvag
  Running command git clone -q https://github.com/paoloripamonti/face-recognition /tmp/pip-req-build-6dd2fvag
  Created wheel for facenet-face-recognition: filename=facenet_face_recognition-0.1-cp36-none-any.whl size=5969 sha256=e1de5d43c0edf0fcc73b66b42652f05fd035ca49f3d0c180ec464cda7eb9b131
  Stored in directory: /tmp/pip-ephem-wheel-cache-ycurzcmx/wheels/b4/0b/30/4413165ce9aaebf21fd5920f82bfc94ab6cf8fdb3dd543bf12
Successfully built facenet-face-recognition


In [ ]:
from sklearn.model_selection import train_test_split
trainx, valid = train_test_split(dataset, test_size=0.1, random_state=42, shuffle=True)

In [30]:
fr = FaceRecognitionClass()
fr.fit_from_dataframe(trainx)

In [31]:
valid_test, valid_pred, valid_scores = [],[],[]
for idx in range(len(valid)):
    path = valid.path.iloc[idx]
    result = fr.predict(path, 0.6)
    for prediction in result["predictions"]:
        valid_pred.append(prediction["person"])
        valid_scores.append(prediction["confidence"])
        valid_test.append(valid.person.iloc[idx])

In [32]:
print("Train Accuracy: %f" % accuracy_score(valid_test, valid_pred))

Train Accuracy: 0.557745


In [33]:
#Using the Face Recognition Algorithms Prediction Function to predict the faces

y_test, y_pred, y_scores = [],[],[]
for idx in range(len(test_dataset)):
    path = test_dataset.path.iloc[idx]
    result = fr.predict(path, 0.6)
    for prediction in result["predictions"]:
        y_pred.append(prediction["person"])
        y_scores.append(prediction["confidence"])
        y_test.append(test_dataset.person.iloc[idx])

In [34]:
print(classification_report(y_test, y_pred))

                                     precision    recall  f1-score   support

                            AJ_Cook       1.00      1.00      1.00         1
                           AJ_Lamas       0.00      0.00      0.00         1
                      Aaron_Eckhart       0.00      0.00      0.00         1
                        Aaron_Guiel       0.00      0.00      0.00         1
                    Aaron_Patterson       0.00      0.00      0.00         1
                      Aaron_Peirsol       1.00      1.00      1.00         1
                         Aaron_Pena       1.00      1.00      1.00         1
                       Aaron_Sorkin       0.00      0.00      0.00         1
                       Aaron_Tippin       1.00      1.00      1.00         1
                          Abba_Eban       0.00      0.00      0.00         1
                   Abbas_Kiarostami       0.00      0.00      0.00         1
                Abdel_Aziz_Al-Hakim       1.00      1.00      1.00         

In [35]:
#Printing the accuracy of prediction

accuracy = accuracy_score(y_test, y_pred)

print("Accuracy: %f" % (accuracy))

Accuracy: 0.639962
